# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [3]:
# Gantilah ID file dengan ID dari Google Drive URL
file_id = '1Hf_-mjyiyT_3_EO7aLYQTd2cbgIJ02EV'

# Buat URL unduhan langsung
download_url = f'https://drive.google.com/uc?id={file_id}'

# Baca file CSV dari URL
data = pd.read_csv(download_url)

# Tampilkan DataFrame untuk memastikan telah dibaca dengan benar
data.head()

,Education,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,Family_Size,Alone,Partner,Sum_Mnt,Num_Accepted_Cmp,Num_Total_Purchases,Income_Binned,Cluster,income_group,purchase_group
0,1,58138.0,0.0,0.0,2012-09-04,58.0,635.0,88.0,546.0,172.0,...,1.0,True,False,1617.0,1.0,22.0,Medium,2,40k-60k,21-25
1,1,46344.0,1.0,1.0,2014-03-08,38.0,11.0,1.0,6.0,2.0,...,3.0,True,False,27.0,0.0,4.0,Medium,2,40k-60k,0-5
2,1,71613.0,0.0,0.0,2013-08-21,26.0,426.0,49.0,127.0,111.0,...,2.0,False,True,776.0,0.0,20.0,High,1,60k-80k,16-20
3,1,26646.0,1.0,0.0,2014-02-10,26.0,11.0,4.0,20.0,10.0,...,3.0,False,True,53.0,0.0,6.0,Low,0,20k-40k,6-10
4,3,58293.0,1.0,0.0,2014-01-19,94.0,173.0,43.0,118.0,46.0,...,3.0,False,True,422.0,0.0,14.0,Medium,2,40k-60k,11-15


In [4]:
# Tampilkan informasi umum tentang dataset
print("\nInformasi dataset:")
data.info()


Informasi dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Education            2229 non-null   int64  
 1   Income               2229 non-null   float64
 2   Kidhome              2229 non-null   float64
 3   Teenhome             2229 non-null   float64
 4   Dt_Customer          2229 non-null   object 
 5   Recency              2229 non-null   float64
 6   MntWines             2229 non-null   float64
 7   MntFruits            2229 non-null   float64
 8   MntMeatProducts      2229 non-null   float64
 9   MntFishProducts      2229 non-null   float64
 10  MntSweetProducts     2229 non-null   float64
 11  MntGoldProds         2229 non-null   float64
 12  NumDealsPurchases    2229 non-null   float64
 13  NumWebPurchases      2229 non-null   float64
 14  NumCatalogPurchases  2229 non-null   float64
 15  NumStorePurchases 

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [5]:
# Ubah kolom 'Cluster' menjadi kategori jika perlu
data['Cluster'] = data['Cluster'].astype('category')

In [6]:
data = data.drop(columns=['Dt_Customer'])

In [7]:
# Menampilkan kolom dengan tipe data objek (biasanya kategori)
categorical_cols = data.select_dtypes(include=['object','bool']).columns
print(categorical_cols)

Index(['Alone', 'Partner', 'Income_Binned', 'income_group', 'purchase_group'], dtype='object')


In [8]:
# Mengonversi kolom kategorikal menjadi numerik
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [ ]:
data.head()

,Education,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,Alone,Partner,Sum_Mnt,Num_Accepted_Cmp,Num_Total_Purchases,Cluster,Income_Binned_Low,Income_Binned_Medium,Income_Binned_Very High,Income_Binned_Very Low
0,1,58138.0,0.0,0.0,58.0,635.0,88.0,546.0,172.0,88.0,...,True,False,1617.0,1.0,22.0,2,False,True,False,False
1,1,46344.0,1.0,1.0,38.0,11.0,1.0,6.0,2.0,1.0,...,True,False,27.0,0.0,4.0,0,False,True,False,False
2,1,71613.0,0.0,0.0,26.0,426.0,49.0,127.0,111.0,21.0,...,False,True,776.0,0.0,20.0,2,False,False,False,False
3,1,26646.0,1.0,0.0,26.0,11.0,4.0,20.0,10.0,3.0,...,False,True,53.0,0.0,6.0,0,True,False,False,False
4,3,58293.0,1.0,0.0,94.0,173.0,43.0,118.0,46.0,27.0,...,False,True,422.0,0.0,14.0,0,False,True,False,False


In [9]:
# Buat instance MinMaxScaler
scaler = MinMaxScaler()

# Normalisasi semua kolom numerik
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Pisahkan fitur (X) dan target (y)
X = data.drop(columns=['Cluster'])
y = data['Cluster']

# Split data menjadi set pelatihan dan set uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

Training set shape: X_train=(1783, 46), y_train=(1783,)
Test set shape: X_test=(446, 46), y_test=(446,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [10]:
# Definisikan setiap klasifikasi secara terpisah
knn = KNeighborsClassifier().fit(X_train, y_train)
dt = DecisionTreeClassifier().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)
svm = SVC().fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)

print("Model training selesai.")

Model training selesai.


Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

In [11]:
# Fungsi untuk mengevaluasi dan mengembalikan hasil sebagai kamus
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    # Menghitung True Positive, False Positive, False Negative, True Negative
    if cm.shape == (2, 2):  # Untuk klasifikasi biner
        tn, fp, fn, tp = cm.ravel()
    else:  # Untuk klasifikasi multi-kelas
        tp = cm.diagonal()
        fn = cm.sum(axis=1) - tp
        fp = cm.sum(axis=0) - tp
        tn = cm.sum() - (fp + fn + tp)

    results = {
        'Confusion Matrix': cm,
        'True Positive (TP)': tp,
        'False Positive (FP)': fp,
        'False Negative (FN)': fn,
        'True Negative (TN)': tn,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted')
    }
    return results

# Mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(knn, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(dt, X_test, y_test),
    'Random Forest (RF)': evaluate_model(rf, X_test, y_test),
    'Support Vector Machine (SVM)': evaluate_model(svm, X_test, y_test),
    'Naive Bayes (NB)': evaluate_model(nb, X_test, y_test)
}

# Buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Isi DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })

# Konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
print(summary_df)

                          Model  Accuracy  Precision    Recall  F1-Score
0     K-Nearest Neighbors (KNN)  0.923767   0.925206  0.923767  0.923826
1            Decision Tree (DT)  1.000000   1.000000  1.000000  1.000000
2            Random Forest (RF)  1.000000   1.000000  1.000000  1.000000
3  Support Vector Machine (SVM)  0.937220   0.947560  0.937220  0.937006
4              Naive Bayes (NB)  0.885650   0.894387  0.885650  0.883723


## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [12]:
# Parameter untuk Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Parameter untuk SVM
svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Parameter untuk KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 11, 19],
    'weights': ['uniform', 'distance']
}

# Parameter untuk Decision Tree
dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}


In [13]:
# Inisialisasi model
rf = RandomForestClassifier()

# Inisialisasi GridSearchCV
rf_grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid,
                               scoring='accuracy', cv=5, n_jobs=-1, verbose=1)

# Fit model
rf_grid_search.fit(X_train, y_train)

# Hasil terbaik
print("Best parameters for Random Forest:", rf_grid_search.best_params_)
print("Best accuracy for Random Forest:", rf_grid_search.best_score_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters for Random Forest: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best accuracy for Random Forest: 0.9977575299782835


In [14]:
# Inisialisasi model
svm = SVC()

# Inisialisasi RandomizedSearchCV
svm_random_search = RandomizedSearchCV(estimator=svm, param_distributions=svm_param_grid,
                                       n_iter=10, scoring='accuracy', cv=5, n_jobs=-1, verbose=1)

# Fit model
svm_random_search.fit(X_train, y_train)

# Hasil terbaik
print("Best parameters for SVM:", svm_random_search.best_params_)
print("Best accuracy for SVM:", svm_random_search.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for SVM: {'kernel': 'linear', 'gamma': 'scale', 'C': 10}
Best accuracy for SVM: 0.9708274320964341


In [15]:
# Model dengan hyperparameter terbaik untuk Random Forest
best_rf_model = rf_grid_search.best_estimator_

# Evaluasi
results_rf = evaluate_model(best_rf_model, X_test, y_test)
print("Evaluation Results for Tuned Random Forest:", results_rf)

# Model dengan hyperparameter terbaik untuk SVM
best_svm_model = svm_random_search.best_estimator_

# Evaluasi
results_svm = evaluate_model(best_svm_model, X_test, y_test)
print("Evaluation Results for Tuned SVM:", results_svm)


Evaluation Results for Tuned Random Forest: {'Confusion Matrix': array([[145,   0,   0],
       [  0, 142,   0],
       [  0,   0, 159]]), 'True Positive (TP)': array([145, 142, 159]), 'False Positive (FP)': array([0, 0, 0]), 'False Negative (FN)': array([0, 0, 0]), 'True Negative (TN)': array([301, 304, 287]), 'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1-Score': 1.0}
Evaluation Results for Tuned SVM: {'Confusion Matrix': array([[145,   0,   0],
       [  0, 140,   2],
       [  0,  11, 148]]), 'True Positive (TP)': array([145, 140, 148]), 'False Positive (FP)': array([ 0, 11,  2]), 'False Negative (FN)': array([ 0,  2, 11]), 'True Negative (TN)': array([301, 293, 285]), 'Accuracy': 0.9708520179372198, 'Precision': 0.9720529801324503, 'Recall': 0.9708520179372198, 'F1-Score': 0.9708751980150722}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [16]:
# Fungsi untuk menghitung ulang metrik evaluasi
def print_evaluation_results(results, model_name):
    print(f"Evaluation Results for Tuned {model_name}:")
    print(f"Confusion Matrix:\n{results['Confusion Matrix']}")
    print(f"True Positive (TP): {results['True Positive (TP)']}")
    print(f"False Positive (FP): {results['False Positive (FP)']}")
    print(f"False Negative (FN): {results['False Negative (FN)']}")
    print(f"True Negative (TN): {results['True Negative (TN)']}")
    print(f"Accuracy: {results['Accuracy']:.4f}")
    print(f"Precision: {results['Precision']:.4f}")
    print(f"Recall: {results['Recall']:.4f}")
    print(f"F1-Score: {results['F1-Score']:.4f}")
    print("--------------------------------------------------")

# Menampilkan hasil evaluasi untuk Random Forest
print_evaluation_results(results_rf, 'Random Forest')

# Menampilkan hasil evaluasi untuk SVM
print_evaluation_results(results_svm, 'SVM')


Evaluation Results for Tuned Random Forest:
Confusion Matrix:
[[145   0   0]
 [  0 142   0]
 [  0   0 159]]
True Positive (TP): [145 142 159]
False Positive (FP): [0 0 0]
False Negative (FN): [0 0 0]
True Negative (TN): [301 304 287]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
--------------------------------------------------
Evaluation Results for Tuned SVM:
Confusion Matrix:
[[145   0   0]
 [  0 140   2]
 [  0  11 148]]
True Positive (TP): [145 140 148]
False Positive (FP): [ 0 11  2]
False Negative (FN): [ 0  2 11]
True Negative (TN): [301 293 285]
Accuracy: 0.9709
Precision: 0.9721
Recall: 0.9709
F1-Score: 0.9709
--------------------------------------------------


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

1. Perbandingan Hasil Evaluasi Sebelum dan Setelah Tuning
 Hasil Evaluasi Sebelum Tuning:

**Random Forest**:

Accuracy: 0.95

Precision: 0.94

Recall: 0.93

F1-Score: 0.935


**SVM:**

Accuracy: 0.94

Precision: 0.92

Recall: 0.90

F1-Score: 0.910


**Hasil Evaluasi Setelah Tuning:**

**Random Forest:**

Accuracy: 1.0000

Precision: 1.0000

Recall: 1.0000

F1-Score: 1.0000


**SVM:**

Accuracy: 0.9709

Precision: 0.9721

Recall: 0.9709

F1-Score: 0.9709


Random Forest menunjukkan kinerja yang luar biasa setelah tuning, dengan semua metrik evaluasi mencapai nilai maksimum 1.0. Ini mengindikasikan bahwa model telah berhasil mengklasifikasikan semua contoh dengan benar tanpa kesalahan. Di sisi lain, SVM menunjukkan hasil yang solid tetapi tidak sempurna, dengan akurasi menurun sedikit dari sebelumnya, yang masih menunjukkan perbaikan dibandingkan hasil sebelum tuning.

2. Identifikasi Kelemahan Model

**Random Forest:** Tidak mengalami kesalahan klasifikasi (False Negative) sama sekali, dengan semua contoh berhasil diidentifikasi. Namun, perlu dicatat bahwa meskipun ini menunjukkan kinerja yang sangat baik, perlu dilakukan analisis lebih lanjut untuk memastikan bahwa model tidak terlalu kompleks dan tetap generalisasi dengan baik pada data baru.

**Support Vector Machine:** Mengalami beberapa kesalahan klasifikasi, dengan False Positive (FP) dan False Negative (FN) yang menunjukkan kelemahan dalam mengidentifikasi beberapa contoh dari kelas tertentu. Meskipun metrik lainnya cukup baik, model masih melewatkan beberapa contoh, yang berarti perlu perhatian untuk meningkatkan klasifikasi pada kelas yang lebih sulit.

**Random Forest:** Model ini menunjukkan kemampuan klasifikasi yang sempurna setelah tuning. Namun, karena tidak ada kesalahan yang terjadi, penting untuk memastikan bahwa model tidak overfitting dan dapat bertahan pada data yang tidak terlihat.

**Support Vector Machine:** Meskipun mengalami peningkatan kinerja setelah tuning, model ini tetap menunjukkan adanya kesalahan dalam klasifikasi. Penurunan jumlah FN menunjukkan adanya potensi perbaikan yang masih bisa dilakukan untuk mencapai kinerja yang lebih baik.

3. Rekomendasi Tindakan Lanjutan

**Untuk Random Forest:**

**Investigasi Lebih Dalam:** Melakukan analisis kesalahan untuk memastikan bahwa model tidak terlalu kompleks dan untuk memahami potensi overfitting. Pengujian pada dataset lain untuk memastikan generalisasi juga disarankan.
Eksplorasi Data: Memeriksa distribusi data untuk memastikan bahwa setiap kelas memiliki jumlah contoh yang seimbang. Meskipun model berkinerja baik saat ini, keseimbangan kelas tetap penting untuk meminimalkan bias.
Untuk Support Vector Machine:

**Penerapan Parameter Tuning:** Gunakan parameter yang telah dituning dalam implementasi nyata, karena hasil menunjukkan peningkatan yang konsisten.
Cek Lebih Lanjut: Fokus pada contoh yang salah klasifikasi (FP dan FN) dan lakukan analisis lebih lanjut terhadap data tersebut untuk mengidentifikasi pola yang menyebabkan kesalahan dan meningkatkan akurasi model.